In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
class random_massing:
    def __init__(self,grid_x,grid_y,grid_z,num_of_use, area_u, area_v):
        self.grid_x = grid_x
        self.grid_y=grid_y
        self.grid_z= grid_z
        self.num_of_use= num_of_use #최대 5개
        self.area_u=area_u #meter
        self.area_v= area_v #meter
    
    #grid_x, grid_y, grid_z 크기의 그리드 생성
    def make_grid(self):
        self.x, self.y, self.z = np.indices((self.grid_x, self.grid_y, self.grid_z))
    
    #각 용도별 구획 크기 입력
    def input_use_size(self):
        self.input_use=[]
        for _ in range(self.num_of_use):
            print('insert 각 용도 별 구획')
            size= int(input())
            self.input_use.append(size)
            
    
    #각 구획의 용도마다 맞는 셀 크기 계산
    def cal_cubenum(self, num , area_u, area_v):
        return num//(area_u*area_v)
    
    #각 용도마다 셀 개수 배정
    def cubenum(self):
        self.cube_num=[]
        for _ in range(self.num_of_use):
            self.cube_num.append(self.cal_cubenum(self.input_use[_],self.area_u, self.area_v))
            
        if sum(self.cube_num)>self.grid_x*self.grid_y*self.grid_z:
            print('size over') #사이즈 과다. 용도 별 영역 크기 줄여야함
            exit(1)
        print(self.cube_num)
        
    def calc_parameter(self):
        #일조 파라메터
        sunarr2=[0 for _ in range(self.grid_x*self.grid_y*self.grid_z)]
        xleft=0
        xright=self.grid_x
        yleft=0
        yright=self.grid_y
    
        while (xleft<xright) & (yleft<yright):
            sun= (xleft<self.x) & (self.x<xright) & (yleft<self.y) & (self.y<yright)
            sunarr= np.array(sun).reshape(-1)
            for index, item in enumerate(sunarr):
                if item==True:
                    sunarr2[index]+=1
            xleft+=1
            xright-=1
            yleft+=1
            yright-=1
                
        #보행자 파라메터
        pedarr2=[0 for _ in range(self.grid_x*self.grid_y*self.grid_z)]
        high=self.grid_z//3+1
        while high>0:
            ped= (self.z<high)
            pedarr= np.array(ped).reshape(-1)
            for index, item in enumerate(pedarr):
                if item==True:
                    pedarr2[index]+=1
            high-=1
            
        
        #뷰 파라메터
        viewarr2=[0 for _ in range(self.grid_x*self.grid_y*self.grid_z)]
        high=3
        while high<9:
            view= (self.z> high)
            viewarr= np.array(view).reshape(-1)
            for index, item in enumerate(viewarr):
                if item == True:
                    viewarr2[index]+=1
            high+=1

        df= pd.DataFrame([sunarr2, pedarr2,viewarr2])
        df=df.transpose()

        scaler = preprocessing.MinMaxScaler()
        df[[0,1,2]] = scaler.fit_transform(df[[0,1,2]])

        estimator = KMeans(self.num_of_use)
        ids = estimator.fit_predict(df)
        
        return ids
        
        
    #default 영역 설정
    def assign_default(self):
        
        self.voxels=[]
        self.colors=[]
        chart=['blue','green','red','yellow','purple','orange','pink']
        
        ids= self.calc_parameter()
        print(ids)
        
        for id in range(self.num_of_use):
            place=[0 for _ in range(self.grid_x*self.grid_y*self.grid_z)]
            for index,item in enumerate(ids):
                if item==id:
                    place[index]=True
                else:
                    place[index]=False
            voxel=np.array(place).reshape(self.grid_x,self.grid_y,self.grid_z)
            color= np.empty(voxel.shape, dtype=object)
            color[voxel]=chart[id]
            self.voxels.append(voxel)
            self.colors.append(color)
        
        
        '''
        #place1은 일조나쁨, 사람접근성나쁨, 차량접근성나쁨
        self.place_1= (0<self.x) & (self.x<self.grid_x-1) & (0<self.y) & (self.y<self.grid_y-1) &(self.z>self.grid_z//3+1)
        voxels1 = self.place_1
        colors1 = np.empty(voxels1.shape, dtype=object)
        colors1[self.place_1] = 'blue'
        
        #place2는 일조나쁨, 사람접근성나쁨, 차량접근성좋음
        self.place_2= (((0<self.x) & (self.x<self.grid_x -1) & (0<self.y) & (self.y<self.grid_y -1)) | (self.z<self.grid_z//3+1)) & (self.z<2*self.grid_z//3-1)
        voxels2 = self.place_2
        colors2 = np.empty(voxels2.shape, dtype=object)
        colors2[self.place_2] = 'green'
        
        #place3=1.일조좋음 2.사람접근성좋음 3.차량접근성좋음
        self.place_3= ((((self.x<1) | (self.x>self.grid_x-2)) | ((self.y<1) | (self.y>self.grid_y-2))) & ((self.z<2*self.grid_z//3) & (self.z>self.grid_z//3-1))) | ((self.z<self.grid_z//3+1) & (self.z>self.grid_z//3-1))
        voxels3 = self.place_3
        colors3 = np.empty(voxels3.shape, dtype=object)
        colors3[self.place_3] = 'red'
        
        #place4 : 1.일조좋음 2.사람접근성좋음 3.차량접근성나쁨
        self.place_4= (((self.x<1) | (self.x>self.grid_x-2)) | ((self.y<1) | (self.y>self.grid_y-2))) & ((self.z>2*self.grid_z//3-1) & (self.z<self.grid_z-1))
        voxels4=self.place_4
        colors4 = np.empty(voxels4.shape, dtype=object)
        colors4[self.place_4] = 'yellow'
        
        #place5 : 1.일조중간 2.사람접근성중간 3.차량접근성나쁨
        self.place_5= (self.z>2*self.grid_z//3)
        voxels5=self.place_5
        colors5 = np.empty(voxels5.shape, dtype=object)
        colors5[self.place_5] = 'purple'
        
        
        self.voxels.append(voxels1)
        self.voxels.append(voxels2)
        self.voxels.append(voxels3)
        self.voxels.append(voxels4)
        self.voxels.append(voxels5)
        
        self.colors.append(colors1)
        self.colors.append(colors2)
        self.colors.append(colors3)
        self.colors.append(colors4)
        self.colors.append(colors5)
        '''
        self.visualize()
   
    #시각화  
    def visualize(self):
        
        fig = plt.figure(figsize=(18,10))
        ax = fig.gca(projection='3d')
        for _ in range(len(self.voxels)):
            vox=ax.voxels(self.voxels[_], facecolors=self.colors[_], edgecolor='k', alpha=0.5)
            
        plt.show()
        
    
    def assign_cell(self):
        color=['blue','green','red','yellow','purple','orange','pink']
        
        for index in range(self.num_of_use-1):
            
            futnparr= self.move_random(index+1) #움직일 x,y,z self.nparr에 저장
            
            if index==0:
                curnparr=self.move_random(index)
            
            if len(np.where(self.voxels[index]==True)[0]) > self.cube_num[index]:
                for i in curnparr[0][self.cube_num[index]:]:
                    for j in curnparr[1][self.cube_num[index]:]:
                        for k in curnparr[2][self.cube_num[index]:]:
                            self.voxels[index][i][j][k]=False
                            self.voxels[index+1][i][j][k]=True
            else:
                for i in futnparr[0][:self.cube_num[index]-len(curnparr[0])]:
                    for j in futnparr[1][:self.cube_num[index]-len(curnparr[0])]:
                        for k in futnparr[2][:self.cube_num[index]-len(curnparr[0])]:
                            self.voxels[index][i][j][k]=True
                            self.voxels[index+1][i][j][k]=False
                            
            curnparr=futnparr
            
            colors1 = np.empty(self.voxels[index].shape, dtype=object)
            colors1[self.voxels[index]] = color[index]
            self.colors[index]=colors1
            
            #개별 시각화
            fig = plt.figure(figsize=(18,10))
            ax = fig.gca(projection='3d')
            vox=ax.voxels(self.voxels[index], facecolors=self.colors[index], edgecolor='k', alpha=0.5)
            plt.show()
        
        colors2 = np.empty(self.voxels[index+1].shape, dtype=object)
        colors2[self.voxels[index+1]] = color[index+1]
        self.colors[index+1]=colors2
        
        #개별 시각화
        fig = plt.figure(figsize=(18,10))
        ax = fig.gca(projection='3d')
        vox=ax.voxels(self.voxels[index+1], facecolors=self.colors[index+1], edgecolor='k', alpha=0.5)
        plt.show()
            
    
    
    def move_random(self, index):
        #방향 바꾸는 블록
        arr=[]
        for i in range(len(np.where(self.voxels[index]==True)[0])):
            arr.append([np.where(self.voxels[index]==True)[0][i]])
            arr[i].append(np.where(self.voxels[index]==True)[1][i])
            arr[i].append(np.where(self.voxels[index]==True)[2][i]) #true 인 place의 (x,y,z)

        seed= np.random.rand(1)[0]
        if seed<0.33:
            arr.sort(key=lambda x:x[2]) #z축
            print('z')
        elif 0.33<=seed and seed<0.66:
            arr.sort(key=lambda x:x[1]) #y축
            print('y')
        else:
            arr.sort(key=lambda x:x[0]) #x축
            print('x')

        nparr=[[],[],[]] #x/y/z 끼리

        for i in range(len(np.where(self.voxels[index]==True)[0])):
            nparr[0].append(arr[i][0])
            nparr[1].append(arr[i][1])
            nparr[2].append(arr[i][2])
        
        return nparr
    

## if __name__ == "__main__":
    #그리드 개수, 용도, 넓이 조정
    rm= random_massing(6,3,9,6,8,8) #2500,2800,2400,2000,600
    rm.input_use_size()
    rm.cubenum()
    rm.make_grid()
    rm.assign_default()
    rm.assign_cell()
    rm.visualize()

In [ ]:
if __name__ == "__main__":
    #그리드 개수, 용도, 넓이 조정
    rm= random_massing(6,3,9,5,8,8)
    rm.input_use_size()
    rm.cubenum()
    rm.make_grid()
    rm.assign_default()
    rm.assign_cell()
    rm.visualize()
    

In [ ]:
rm= random_massing(7,4,8,5,8,8) #2500,2800,2400,1200,800,600
rm.input_use_size()

In [ ]:
rm.cubenum()
rm.make_grid()

In [ ]:
rm.assign_default()

In [ ]:
rm.assign_cell()

In [ ]:
rm.visualize()